In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)

from datasets import Dataset

In [ ]:
import glob
import os
import pandas as pd
path = "14189072"  
pattern = os.path.join(path, "Russia_2_part_*.gzip.parquet")

files = glob.glob(pattern)

dfs = [pd.read_parquet(f, engine="pyarrow") for f in files]
df = pd.concat(dfs, ignore_index=True)

In [7]:
df.to_csv("russian_disinformation_tweets.csv")

In [4]:
tweets = pd.read_csv("russian_disinformation_tweets.csv")


print(len(tweets[tweets["is_control"] == False]))
print(len(tweets[tweets["is_control"] == True]))
tweets = tweets[["post_text", "is_control"]].dropna()
tweets = tweets.rename(columns={"post_text": "text", "is_control": "label"})

920761
90291


In [5]:
train_tweets , eval_tweets  = train_test_split(tweets , test_size=0.1, stratify=tweets["label"], random_state=42)

In [6]:
model_name = "IDA-SERICS/PropagandaDetection"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [7]:
def tokenize_batch(batch):
    return tokenizer(batch["text"], truncation=True, max_length=256)

In [13]:
train_tweets["label"] = train_tweets["label"].astype(int)
eval_tweets["label"]  = eval_tweets["label"].astype(int)

train_ds = Dataset.from_pandas(train_tweets).map(tokenize_batch, batched=True)
eval_ds  = Dataset.from_pandas(eval_tweets).map(tokenize_batch, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/909946 [00:00<?, ? examples/s]

Map:   0%|          | 0/101106 [00:00<?, ? examples/s]

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    p, r, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    return {"accuracy": acc, "precision": p, "recall": r, "f1": f1}


In [15]:
training_args = TrainingArguments(
    output_dir="./propaganda-bert-finetuned",
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

C:\Users\Irene\AppData\Local\Temp\ipykernel_18940\2076841044.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:
trainer.train()

metrics = trainer.evaluate(eval_ds)
print("Evaluation results:", metrics)

from sklearn.metrics import classification_report
preds_output = trainer.predict(eval_ds)
y_true = preds_output.label_ids
y_pred = np.argmax(preds_output.predictions, axis=1)
print(classification_report(y_true, y_pred, digits=4))

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 